In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/b-ner.csv")
data.head()

,Sentence #,Word,Tag
0,Sentence: 1,বাংলাদেশে,B-geo
1,NaN,কক্সবাজার,B-geo
2,NaN,এলাকায়,O
3,NaN,রোহিঙ্গাদের,B-gpe
4,NaN,জন্য,O


In [ ]:
data.drop(columns=["Sentence #"],inplace=True)

In [ ]:
data['Tag'].value_counts()

,count
Tag,
O,246943
B-geo,10421
B-per,9201
I-per,9024
B-org,5465
B-tim,4736
I-org,4322
B-gpe,2453
I-tim,2449


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297417 entries, 0 to 297416
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Word    297417 non-null  object
 1   Tag     297417 non-null  object
dtypes: object(2)
memory usage: 4.5+ MB


In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

# Make sure to download the punkt tokenizer if not already done
nltk.download('punkt')

# Load the dataset
# Assuming the dataset is in a CSV format with columns 'text' and 'tag'
# data = pd.read_csv('bengali_dataset.csv')  # Replace with your dataset path

# Display the first few rows of the dataset
print(data.head())

# Data Cleaning Function
def clean_text(text):
    # Remove unwanted characters, links, etc.
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)  # Remove @mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^a-zA-Z0-9\u0980-\u09FF\s]', '', text)  # Remove non-Bengali characters and special characters
    text = text.strip()  # Remove leading and trailing whitespace
    return text

# Apply cleaning to the 'text' column
data['Word'] = data['Word'].apply(clean_text)

# Tokenization Function
def tokenize_text(text):
    return word_tokenize(text)

# Apply tokenization
data['tokens'] = data['Word'].apply(tokenize_text)

# Display the cleaned and tokenized data
print(data[['Word', 'tokens']].head())

# Prepare data for model training
# Assuming your NER tagging is in a separate column 'tag'
# Flatten the tokens and tags for training
flattened_data = []

for index, row in data.iterrows():
    for token, tag in zip(row['tokens'], row['Tag'].split()):  # Adjust based on your tag format
        flattened_data.append((token, tag))

# Create a DataFrame for the flattened data
flattened_df = pd.DataFrame(flattened_data, columns=['token', 'tag'])

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(flattened_df, test_size=0.2, random_state=42)

# Display the training and testing data
print("Training data:")
print(train_data.head())
print("Testing data:")
print(test_data.head())


          Word    Tag
0    বাংলাদেশে  B-geo
1    কক্সবাজার  B-geo
2      এলাকায়      O
3  রোহিঙ্গাদের  B-gpe
4         জন্য      O


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


          Word         tokens
0    বাংলাদেশে    [বাংলাদেশে]
1    কক্সবাজার    [কক্সবাজার]
2      এলাকায়      [এলাকায়]
3  রোহিঙ্গাদের  [রোহিঙ্গাদের]
4         জন্য         [জন্য]
Training data:
               token    tag
67505            দেব  I-per
105966       জামাইকে      O
209885           করা      O
50173   সুপারিশগুলোর      O
12077            আমু  B-per
Testing data:
                token    tag
210854         গ্রামে      O
158672     রোহিঙ্গারা  B-gpe
171409  কক্সবাজারমুখী  B-geo
109634             শি  B-per
47714          মজুদের      O


In [ ]:
from gensim.models import Word2Vec
import numpy as np

# Create a list of sentences from the tokens
sentences = data['tokens'].tolist()

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to convert tokens to vectors
def vectorize_tokens(tokens):
    vectors = []
    for token in tokens:
        if token in word2vec_model.wv:
            vectors.append(word2vec_model.wv[token])
        else:
            vectors.append(np.zeros(word2vec_model.vector_size))  # Use zero vector for unknown tokens
    return vectors


In [ ]:
train_vectors = train_data['token'].apply(vectorize_tokens).tolist()
test_vectors = test_data['token'].apply(vectorize_tokens).tolist()

In [ ]:
train_data

,token,tag
67505,দেব,I-per
105966,জামাইকে,O
209885,করা,O
50173,সুপারিশগুলোর,O
12077,আমু,B-per
...,...,...
119879,শেকলে,O
259178,নিজেই,O
131932,মুষ্টিমেয়,O
146867,নির্বাচনের,O


In [ ]:
data['Tag'].unique()

array(['B-geo', 'O', 'B-gpe', 'B-per', 'I-per', 'B-tim', 'B-org', 'I-org',
       'B-art', 'I-art', 'I-tim', 'B-eve', 'I-eve', 'I-geo', 'I-gpe',
       'B-nat', 'I-nat'], dtype=object)

In [ ]:
# dict1={'B-geo':1,'O':2,'B-gpe':3,'B-per':4,'I-per':5,'B-org':6,'I-org':7,'I-geo':8,'B-tim':9,'I-tim':10,'I-gpe':11,'B-nat':12,'I-nat':13}
# train_data['Tag']=train_data['tag'].map(dict1)
# test_data['Tag']=test_data['tag'].map(dict1)

In [ ]:
# Step 1: Create a mapping for tags to indices and vice versa
tags = list(data['Tag'].unique())  # Get a list of all unique tags
tag_to_index = {tag: idx for idx, tag in enumerate(tags)}
index_to_tag = {idx: tag for tag, idx in tag_to_index.items()}

# Step 2: Convert tags to integer sequences using the mapping
def convert_tags_to_indices(tags_list):
    return [tag_to_index[tag] for tag in tags_list]

# Convert tags in training and test data to indices
train_data['tag_index'] = train_data['tag'].apply(convert_tags_to_indices)
test_data['tag_index'] = test_data['tag'].apply(convert_tags_to_indices)

# Step 3: Pad the tag sequences to ensure uniform length
y_train_padded = pad_sequences(train_data['tag_index'].tolist(), maxlen=max_length, padding='post', value=tag_to_index['O'])
y_test_padded = pad_sequences(test_data['tag_index'].tolist(), maxlen=max_length, padding='post', value=tag_to_index['O'])

# Step 4: Convert y_train and y_test to one-hot encoded vectors for multi-class classification
y_train = [to_categorical(seq, num_classes=len(tag_to_index)) for seq in y_train_padded]
y_test = [to_categorical(seq, num_classes=len(tag_to_index)) for seq in y_test_padded]

# Convert to numpy arrays for training
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Now, X_train, y_train, X_test, and y_test are ready for model training.


KeyError: 'I'

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Dropout

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape=(max_length, 100)))  # 100 is the word vector size
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(len(tag_to_index), activation='softmax')))  # TimeDistributed for sequence labeling
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, verbose=1)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')